In [10]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import feedparser
import yaml
import re

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException

import time
import pandas as pd

### Extract All The Links

In [11]:
PATH = "C:\Program Files (x86)\web_driver\chromedriver.exe"
driver = webdriver.Chrome(PATH)

driver.get('https://dialnet.unirioja.es/tesis/fechalectura')

In [13]:
%%time
absLinks=[]
max_pages = 0
while True:
    food1 = driver.find_elements_by_css_selector("span.titulo")
    for tags in food1:
        food2 = tags.find_elements_by_tag_name("a")
        for link in food2:
            absLinks.append(link.get_attribute("href"))
    
    # extract links
    try:
        time.sleep(7)
        driver.find_element_by_xpath('//*[@id="pieDeListadoDeBusquedaDeAutores"]/ul/li[4]/a').click() 
        max_pages += 1
        if max_pages == 10:
            break
    # stop if no more pages available
    except NoSuchElementException:
        break


Wall time: 1min 25s


In [14]:
%%time
from tqdm import tqdm
data = []
# this for loop goes through only 5 out of 708 links
for i in absLinks:
    time.sleep(5)
    driver.get(i)
    # may have to use xpath instead
    try:
        contents = driver.find_elements_by_xpath('//*[@id="resumen"]')
        for content in contents:
            language = content.text
        
    except NoSuchElementException:
        break
        

    #collect the data
    lang= {'content': language}
    data.append(lang)

KeyboardInterrupt: 

In [211]:
en_sp = pd.DataFrame(data)

In [212]:
en_sp = en_sp[:174]

In [213]:
# spanish variable
en_sp['content'] = en_sp['content'].apply(lambda x: re.sub(r'\n', '', x))
# only Spanish translations: null (0, nan)

In [214]:
en_sp['Spanish'] = en_sp['content'].apply(lambda x: x if 'Español' in x else 0)

In [215]:
en_sp = en_sp[en_sp['Spanish'] != 0]

In [216]:
en_sp['Spanish_text'] = en_sp['Spanish'].apply(lambda x: x.split('English')[0])
en_sp['English_text'] = en_sp['Spanish'].apply(lambda x: x.split('English')[-1])
en_sp['English_text'] = en_sp['English_text'].apply(lambda x: x.split('Galego')[0] if 'Galego' in x else x)

In [217]:
en_sp.reset_index(inplace=True)

In [218]:
en_sp.head(10)

,index,content,Spanish,Spanish_text,English_text
0,1,EspañolEsta tesis quiere profundizar sobre el ...,EspañolEsta tesis quiere profundizar sobre el ...,EspañolEsta tesis quiere profundizar sobre el ...,This thesis wants to delve into the case of th...
1,2,EspañolLa violencia en sus diversas variantes ...,EspañolLa violencia en sus diversas variantes ...,EspañolLa violencia en sus diversas variantes ...,Violence in its several variations is present ...
2,3,EspañolEsta tesis doctoral es un ensayo clínic...,EspañolEsta tesis doctoral es un ensayo clínic...,EspañolEsta tesis doctoral es un ensayo clínic...,This doctoral thesis is a clinical trial that ...
3,4,EspañolLa frustración es un fenómeno que puede...,EspañolLa frustración es un fenómeno que puede...,EspañolLa frustración es un fenómeno que puede...,Frustration can be mitigated if there has been...
4,5,EspañolLas políticas lingüísticas europeas pro...,EspañolLas políticas lingüísticas europeas pro...,EspañolLas políticas lingüísticas europeas pro...,European language policies promote the trainin...
5,6,EspañolLos pabellones de conveniencia han teni...,EspañolLos pabellones de conveniencia han teni...,EspañolLos pabellones de conveniencia han teni...,Flags of convenience have had a deep impact on...
6,7,EspañolRESUMEN La presente tesis doctoral pret...,EspañolRESUMEN La presente tesis doctoral pret...,EspañolRESUMEN La presente tesis doctoral pret...,ABSTRACT This PhD thesis aims to advance the k...
7,9,"EspañolEn la actualidad, la energía es el prin...","EspañolEn la actualidad, la energía es el prin...","EspañolEn la actualidad, la energía es el prin...","Nowadays, energy is the main backbone of human..."
8,21,El español se habla en más de una veintena de ...,El español se habla en más de una veintena de ...,El español se habla en más de una veintena de ...,El español se habla en más de una veintena de ...
9,22,EspañolLos seres humanos y los animales experi...,EspañolLos seres humanos y los animales experi...,EspañolLos seres humanos y los animales experi...,Humans and animals undergo morphological devel...


if spanish_text == English_text drop

In [219]:
en_sp['Spanish_text'][9]

'EspañolLos seres humanos y los animales experimentan un desarrollo morfológico desde la infancia hasta la edad adulta, que tiene lugar en paralelo con el desa rrollo cognitivo. De hecho, se ha demostrado que este proceso facilita el aprendizaje en los humanos. Sin embargo, la mayoría de los trabajos sobre robótica del desarrollo consideran morfologías fijas, abordando sólo el desarrollo del sistema cognitivo de los robots e ignorando la interacción entre el desarrollo cognitivo y el desarrollo morfológico. Además, los pocos casos encontrados en la literatura en que el desarrollo cognitivo y morfológico se abordan conjunta‐ mente en los sistemas robóticos presentan resultados bastante heterogéneos y no concluyentes en cuanto a las causas y los mecanismos que explican la in fluencia del desarrollo morfológico en el aprendizaje. En este trabajo se aborda el problema de la identificación de los rasgos del desarrollo morfológico que influyen en el aprendizaje, así como el cómo y el porqué 

In [220]:
en_sp['English_text'][9]

'Humans and animals undergo morphological development from infancy to adulthood, which takes place in parallel with cognitive development. In fact, this process has been shown to facilitate learning in humans. However, most of the works on developmental robotics consider fixed morphologies, addressing only the development of the cognitive system of the robots and ignoring the interaction between cognitive development and morphological development. Moreover, the few cases found in the literature where cognitive and morphological development are jointly addressed in robotic systems present rather heterogeneous and inconclusive results regarding the causes and the mechanisms that explain the influence of morphological development on learning. This work addresses the problem of identifying those features of morphological development that influence learning, as well as how and why this influence comes about. To this end, a simple and controlled experimental framework based on legged robot m